# South Korea Qiskit Hackathon - Metal Tutorial

In [1]:
%load_ext autoreload
%autoreload 2

*Make sure to have the right kernel selected!*

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs

%metal_heading Welcome to Qiskit Metal!

Welcome to Qiskit Metal! 

For this example tutorial, we will attempt to create a simple two qubit chip. We will want to generate the layout, simulate/analyze and tune the chip to hit the parameters we are wanting, finally rendering to a GDS file.

One could generate subsections of the layout and tune individual components first, but in this case we will create all of the layout. We will want a transmon (in this case choosing ones with charge lines), meandered and simple transmission lines, capacitor couplers, and launchers for wirebond connections. So we will import these, and also create a design instance and launch the GUI.

### Layout

In [3]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_cl import TransmonPocketCL

from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander
from qiskit_metal.qlibrary.interconnects.pathfinder import RoutePathfinder

from qiskit_metal.qlibrary.passives.cap_three_fingers import CapThreeFingers
from qiskit_metal.qlibrary.passives.launchpad_wb import LaunchpadWirebond

In [4]:
design = metal.designs.DesignPlanar()

gui = metal.MetalGUI(design)

Since we are likely to be making many changes while tuning and modifying our design, we will enable overwriting. We can also check all of the chip properties to see if we want to change the size or any other parameter.

In [5]:
design.overwrite_enabled = True
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

We will add the two qubits. We know we will want each qubit to have two connection pads, one for readout, and another for the connection to the other qubit. We can see any options the qubit qcomponent has to figure out what we might want to modify when creating the component. This will include the components default options (which the component designer included) as well as renderer options (which are added based on what renderers are present in Metal).

In [6]:
TransmonPocketCL.get_template_options(design)

{'pos_x': '0um',
 'pos_y': '0um',
 'connection_pads': {},
 '_default_connection_pads': {'pad_gap': '15um',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '5um',
  'pad_cpw_extent': '25um',
  'cpw_width': 'cpw_width',
  'cpw_gap': 'cpw_gap',
  'cpw_extend': '100um',
  'pocket_extent': '5um',
  'pocket_rise': '65um',
  'loc_W': '+1',
  'loc_H': '+1'},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'orientation': '0',
 'make_CL': True,
 'cl_gap': '6um',
 'cl_width': '10um',
 'cl_length': '20um',
 'cl_ground_gap': '6um',
 'cl_pocket_edge': '0',
 'cl_off_center': '50um',
 'hfss_inductance': '10nH',
 'hfss_capacitance': 0,
 'hfss_resistance': 0,
 'hfss_mesh_kw_jj': 7e-06,
 'q3d_inductance': '10nH',
 'q3d_capacitance': 0,
 'q3d_resistance': 0,
 'q3d_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

We will then add those two qubits with the options we are wanting to define, these can all be modified easily later too.
The rebuild command is included so the changes can be seen immediately in the GUI.

In [7]:
options =  dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    cl_pocket_edge = '180',
    connection_pads=dict(
        readout = dict(loc_W=+1, loc_H=+1),
        bus = dict(loc_W=-1, loc_H=-1, )
    ))
        
Q1 = TransmonPocketCL(design,'Q1', options = dict(
        pos_x='0.7mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_01',
        hfss_inductance ='14nH',
        **options))

Q2 = TransmonPocketCL(design,'Q2', options = dict(
        pos_x='-0.7mm', 
        pos_y='0mm', 
        gds_cell_name ='FakeJunction_02',
        hfss_inductance ='12nH',
        orientation = '180',
        **options))

gui.rebuild()
gui.autoscale()

We will next connect the two transmons together to form a bus using RoutePathfinder, an auto drawing CPW transmission line. We simply have to give the start and end location using the component pins of what we are trying to connect to.
We can use the GUI to confirm the pin names we want to give as inputs.

Pins also play an important role with rendering and simulations, as any unconnected pin can be defined as a short, open, or driven terminal.

In [8]:
bus_Q1_Q2 = RoutePathfinder(design, 'Bus_Q1_Q2', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='250um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='bus'),
                                                end_pin=Dict(
                                                    component='Q2',
                                                    pin='bus')
                                            )))

gui.rebuild()
gui.autoscale()

08:37AM 09s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=trace has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 09s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=cut has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 09s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=trace has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 09s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=cut has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.


Then the readout structures can be added, being the capacitor couplers and meandered transmission lines to form the readout resonators. 

In [9]:
cap_Q1 = CapThreeFingers(design, 'Cap_Q1', options= dict(pos_x='2.5mm', pos_y='0.25mm', orientation='90', finger_length = '40um'))
cap_Q2 = CapThreeFingers(design, 'Cap_Q2', options= dict(pos_x='-2.5mm', pos_y='-0.25mm', orientation='-90', finger_length = '40um'))

gui.rebuild()
gui.autoscale()

08:37AM 09s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=trace has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 09s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=cut has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.


In [10]:
readout_Q1 = RouteMeander(design,'Readout_Q1', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q1',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Cap_Q1',
                                                    pin='a')
                                            ),
                                            lead=Dict(
                                                start_straight='0.325mm',
                                                end_straight = '125um'#put jogs here
                                            ),
                                            meander=Dict(
                                                asymmetry = '-50um'),
                                            fillet = "99um",
                                            total_length = '5mm'))

gui.rebuild()
gui.autoscale()

08:37AM 10s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=trace has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 10s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=cut has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.


In [11]:
readout_Q2 = RouteMeander(design,'Readout_Q2', options = dict( 
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Q2',
                                                    pin='readout'),
                                                end_pin=Dict(
                                                    component='Cap_Q2',
                                                    pin='a')
                                            ),
                                            lead=Dict(
                                                start_straight='0.325mm',
                                                end_straight = '125um'#put jogs here
                                            ),
                                            meander=Dict(
                                                asymmetry = '-50um'),
                                            fillet = "99um",
                                            total_length = '6mm'))

gui.rebuild()
gui.autoscale()

08:37AM 11s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=trace has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 11s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=cut has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.


With the launchers and short transmission lines to connect them to the capacitors and the charge lines.

In [12]:
launch_Q1_read = LaunchpadWirebond(design, 'Launch_Q1_Read', options = dict(pos_x = '3.5mm', orientation = '180'))
launch_Q2_read = LaunchpadWirebond(design, 'Launch_Q2_Read', options = dict(pos_x = '-3.5mm', orientation = '0'))

launch_Q1_cl = LaunchpadWirebond(design, 'Launch_Q1_CL', options = dict(pos_x = '1.35mm', pos_y = '-2.5mm', orientation = '90'))
launch_Q2_cl = LaunchpadWirebond(design, 'Launch_Q2_CL', options = dict(pos_x = '-1.35mm', pos_y = '2.5mm', orientation = '-90'))

gui.rebuild()
gui.autoscale()

08:37AM 12s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=trace has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.
08:37AM 12s WARNING [check_lengths]: For path table, component=Bus_Q1_Q2, key=cut has short segments that could cause issues with fillet. Values in (1-3)  are index(es) in shapley geometry.


In [13]:
tl_Q1 = RoutePathfinder(design, 'TL_Q1', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='150um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Launch_Q1_Read',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Cap_Q1',
                                                    pin='b')
                                            )))

tl_Q2 = RoutePathfinder(design, 'TL_Q2', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='150um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Launch_Q2_Read',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Cap_Q2',
                                                    pin='b')
                                            )))

gui.rebuild()
gui.autoscale()

08:37AM 12s ERROR [rebuild]: ERROR in building component name=TL_Q1, error='NoneType' object has no attribute 'coords'


AttributeError: 'NoneType' object has no attribute 'coords'

In [ ]:
tl_Q1_cl = RoutePathfinder(design, 'TL_Q1_CL', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='150um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Launch_Q1_CL',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Q1',
                                                    pin='Charge_Line')
                                            )))

tl_Q2_cl = RoutePathfinder(design, 'TL_Q2_CL', options = dict(
                                            fillet='99um',
                                            lead=dict(end_straight='150um'),
                                            pin_inputs=Dict(
                                                start_pin=Dict(
                                                    component='Launch_Q2_CL',
                                                    pin='tie'),
                                                end_pin=Dict(
                                                    component='Q2',
                                                    pin='Charge_Line')
                                            )))

gui.rebuild()
gui.autoscale()

### Simulation and Analysis

With our fully designed chip now laid out, we can start to focus on tuning the components to hit the circuit parameters we are interested in. These will tend to be;
- qubit: frequency, the anharmonicity/alpha, and the coupling strength (as chi, g, or other)
- busses: frequency (if resonant), the coupling strength
- readout: frequency, coupling strength, coupling to external lines (as kappa, Q_external, or other)

All of the qubit paramters can initially be tuned via a capacitance matrix and the lumped oscillator method. This analysis is not as accurate as others, but allows for fairly fast and small simulations. We will start setting up this simulation by rendering the qubit of interest into Ansys Q3D.

We first create an instance of a Q3D render, connect it to Ansys and modify the options for the simulation we want.

#### Qubit and LOM

In [ ]:
qhk21_q3d = design.renderers.q3d

We can see what options are directly attached to the q3d renderer and make any changes we may wish, such as increasing the buffer size for the bounding box when simulation subsections of the chip.

In [ ]:
qhk21_q3d.options

In [ ]:
#If you don't already have ansys open; qhk21_q3d.open_ansys() 
qhk21_q3d.connect_ansys()

In [ ]:
qhk21_q3d.activate_q3d_design("Qubit1")
qhk21_q3d.add_q3d_setup(name = 'QubitTune', max_passes = 15, min_converged_passes = 2, percent_error = 0.1)
qhk21_q3d.activate_q3d_setup('QubitTune')

With the design and analysis setup, we render the qubit. For LOM analysis, we want the connection pads to be terminated with opens, so indicate this by stating which unconnected pins should have open terminations.

In [ ]:
qhk21_q3d.render_design(['Q1'], [('Q1', 'readout'), ('Q1', 'bus')])

In [ ]:
qhk21_q3d.analyze_setup("QubitTune")

Once complete, we can grab the capacitance matrix and/or call on an LOM analysis of the simulation;

In [ ]:
qhk21_q3d.get_capacitance_matrix()

In [ ]:
#dict_lom = fourq_q3d.lumped_oscillator_vs_passes(Lj, Cj ~ 2 fF, N- total number of connectionPads, fr (readout frequency),
# [fb1,fb2,.... fbN-1] - list of the bus frequencies, maxPass - how many passes did Ansys Q3D take)
# In our case, the last element in the bus list refers to the charge line

dict_lom = qhk21_q3d.lumped_oscillator_vs_passes(14, 2, 3, 7, [0.1, 0.1], 9)

In [ ]:
qhk21_q3d.plot_convergence_main(dict_lom);
qhk21_q3d.plot_convergence_chi(dict_lom)

Looking at the convergences, we can see that we did not have enough passes for the simulation. In this case, we want to modify our simulation setup, 'QubitTune', to improve our convergence.

Using these results, we can make appropriate changes to our qubit layout. Say our anharmonicity/alpha is higher than we want? Since we know Ec~ 1/C, we can change parts of the transmon pocket to lower the total capacitance seen across the junction.
If g to the bus is too low, we can increase the width of the connection pad so that the capacitance between the connection pad and charge island is greater.

One must also keep in mind that any such changes will impact the other parameters, so careful tweaks and iterations of simulation/analysis often end up being necessary.

If wanting to make changes and re-render your design, you first should clear your current design.

In [ ]:
if qhk21_q3d.pinfo is not None:
    obj_names = qhk21_q3d.pinfo.get_all_object_names()
    if obj_names:
        qhk21_q3d.clean_active_design()

When done with all the simulations, you can disconnect from Ansys EDT.

In [ ]:
qhk21_q3d.disconnect_ansys()

#### Qubits and EPR

Now, although the previous simulation and analysis captured all of the parameters of the qubits and coupling to the bus, a more accurate (all be it slower) approach is to render the qubits and their coupling into an eigenmode simulation and perform EPR analysis on the result.

We again first setup an instance of an eigenmode render;

In [ ]:
qhk21_ehfss = design.renderers.hfss

In [ ]:
qhk21_ehfss.connect_ansys()

In [ ]:
qhk21_ehfss.activate_eigenmode_design("Q1_Q2_Bus")

In [ ]:
qhk21_ehfss.add_eigenmode_setup(name='QubitTune', 
                                min_freq_ghz=3, 
                                n_modes=3, 
                                max_passes=11)
qhk21_ehfss.activate_eigenmod_setup('QubitTune')

In this instance an analysis setup was automatically added, so we can just access that one and make the changes to the simulation that we want.

In [ ]:
e_setup = qhk21_ehfss.pinfo.setup
e_setup.passes = 12
e_setup.n_modes = 2
e_setup.max_delta_f = 0.1
e_setup.min_converged = 2

Then render both qubits, and the coupling transmission line. We leave the readout connection pads and charge lines shorted.

In [ ]:
qhk21_ehfss.render_design(['Bus_Q1_Q2', 'Q1', 'Q2'], [])

Design variables can also be added in for direct simulation sweeps.

In [ ]:
e_design = qhk21_ehfss.pinfo.design
e_design.set_variable('Lj1', '14 nH')
e_design.set_variable('Cj1', '0 fF')
e_design.set_variable('Lj2', '12 nH')
e_design.set_variable('Cj2', '0 fF')

In [ ]:
qhk21_ehfss.analyze_setup("QubitTune")

Once completed, we can check the convergence to see if more passes might be necessary.

In [ ]:
qhk21_ehfss.plot_convergences()

This eigenmode simulation is also a quick method to check for any bus or readout resonator frequencies. 
If happy with the simulation, we can then jump to some EPR analysis

In [ ]:
import pyEPR as epr

In [ ]:
pinfo = qhk21_ehfss.pinfo
pinfo.junctions['jj1'] = {'Lj_variable': 'Lj1', 'rect': 'JJ_rect_Lj_Q1_rect_jj', 
                             'line': 'JJ_Lj_Q1_rect_jj_',  'Cj_variable': 'Cj1'}
pinfo.junctions['jj2'] = {'Lj_variable': 'Lj2', 'rect': 'JJ_rect_Lj_Q2_rect_jj', 
                             'line': 'JJ_Lj_Q2_rect_jj_',  'Cj_variable': 'Cj2'}
pinfo.validate_junction_info() # Checks that valid names of variables and objects have been supplied

#Specifying the dissipative elements
pinfo.dissipative['dielectrics_bulk']    = ['main']

eprd = epr.DistributedAnalysis(pinfo)

We can first look at the electric field and subtrate participation.

In [ ]:
eprd.set_mode(1)
ℰ_elec = eprd.calc_energy_electric()
ℰ_elec_substrate = eprd.calc_energy_electric(None, 'main')
ℰ_mag = eprd.calc_energy_magnetic()

print(f"""
ℰ_elec_all       = {ℰ_elec}
ℰ_elec_substrate = {ℰ_elec_substrate}
EPR of substrate = {ℰ_elec_substrate / ℰ_elec * 100 :.1f}%

ℰ_mag_all       = {ℰ_mag}
ℰ_mag % of ℰ_elec_all  = {ℰ_mag / ℰ_elec * 100 :.1f}%
""")

In [ ]:
eprd.do_EPR_analysis()

epra = epr.QuantumAnalysis(eprd.data_filename)
epra.analyze_all_variations(cos_trunc = 7, fock_trunc = 6)

swp_variable = 'Lj1' # suppose we swept an optimetric analysis vs. inductance Lj
epra.plot_hamiltonian_results(swp_variable=swp_variable)
epra.report_results(swp_variable=swp_variable, numeric=True)

From the analysis results we can determine the qubits anharmonicities and coupling strength.

Other analysis is still being added, such as the impedance analysis, though some already be done manually by renderering to a driven modal simulation and performing frequency sweeps to extract the S-Parameters or Impedance matrix. This would be an easy way to, say, determine the external quality factor of a readout resonator. *See guide 6-Analysis*

Once the analysis and tuning is complete, we can disconnect from Ansys EDT.

In [ ]:
qhk21_ehfss.disconnect_ansys()

### Rendering to a GDS File

Once all of the tuning is complete, we will want to prepare a GDS file so we can create a mask and fabricate our chip. We first create a gds render instance.

In [ ]:
qhk21_gds = design.renderers.gds

The various options for the gds renderer can also be checked and changed as necessary. A key option is the gds file which holds the cells for your junction ebeam design. Make sure this is pointing at the correct file so they are placed in your final mask at the appropriate locations.

In [ ]:
qhk21_gds.options

In [ ]:
qhk21_gds.options['path_filename'] = '../../resources/Fake_Junctions.GDS'
qhk21_gds.options['no_cheese']['buffer']='50um'

In [ ]:
qhk21_gds.export_to_gds('QHK21_Tutorial.gds')

Now that the design is finished, we can close the GUI.

In [ ]:
gui.main_window.close()